In [ ]:
import os
import csv
import numpy as np
import sklearn
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn import datasets
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.decomposition import PCA


In [ ]:
%matplotlib inline

#read gender data
with open(os.getcwd()  + '/gender1.csv') as file:
    #if not working, replace with with open(os.getcwd()  + '\\krkopt.csv') as file:
    csv_input = csv.reader(file, delimiter=',')
    data_i = []
    for i, row in enumerate(csv_input):
        data_i.append(list(map(float,row)))
    gender_data = np.array(data_i)
    l=len(gender_data[0])
    x_init=gender_data[:,0:l-1]
    y_init=gender_data[:,l-1].astype(int)
X = pd.DataFrame(x_init)
X.columns=['hair','forehead_w','forehead_h','nose_w','nose_l','lips','Distance']
y = pd.DataFrame(y_init)
y.columns = ['Targets']
print(X)
print(y)

In [ ]:
#read gender data
with open(os.getcwd()  + '/star.csv') as file:
    #if not working, replace with with open(os.getcwd()  + '\\krkopt.csv') as file:
    csv_input = csv.reader(file, delimiter=',')
    data_i = []
    for i, row in enumerate(csv_input):
        data_i.append(list(map(float,row)))
    gender_data = np.array(data_i)
    l=len(gender_data[0])
    x_init=gender_data[:,0:l-1]
    y_init=gender_data[:,l-1].astype(int)
X = pd.DataFrame(x_init)
#X.columns=['hair','forehead_w','forehead_h','nose_w','nose_l','lips','Distance']
X.columns=['Vmag','Plx','e_Plx','B-V','Amag']
y = pd.DataFrame(y_init)
y.columns = ['Targets']
print(X)
print(y)

In [ ]:
from scipy.spatial.distance import cdist
distortions = []
inertias = []
mapping1 = {}
mapping2 = {}
K = range(1, 15)
 
for k in K:
    # Building and fitting the model
    kmeanModel = KMeans(n_clusters=k).fit(X)
    kmeanModel.fit(X)
 
    distortions.append(sum(np.min(cdist(X, kmeanModel.cluster_centers_,
                                        'euclidean'), axis=1)) / X.shape[0])
    inertias.append(kmeanModel.inertia_)
 
    mapping1[k] = sum(np.min(cdist(X, kmeanModel.cluster_centers_,
                                   'euclidean'), axis=1)) / X.shape[0]
    mapping2[k] = kmeanModel.inertia_

In [ ]:
plt.plot(K, distortions, 'bx-')
plt.xlabel('Values of K')
plt.ylabel('Distortion')
plt.title('The Elbow Method using Distortion')
plt.show()

In [ ]:
# Build the KMeans Model
kmeans = KMeans(n_clusters=8)
clusters = kmeans.fit_predict(X)
print(clusters)
from scipy.stats import mode
labels = np.zeros_like(clusters)

for i in range(8):
    cat = (clusters == i)
    #labels[cat] = mode(y.Targets[cat])[0]
print(labels)
    
acc = accuracy_score(y, labels)
print('Accuracy = ', acc)
plt.figure(figsize=(12,12))

colormap = np.array(['red',  'blue','green','yellow','#fc03d3','#44cee3','#eb9d28','#a39176'])

# Plot the Original Classifications using Petal features
#plt.subplot(2, 2, 1)
#plt.scatter(X.forehead_w, X.forehead_h, c = colormap[y.Targets], s=40)
#plt.scatter(X.Vmag, X.Plx, c = colormap[y.Targets], s=40)
#plt.title('Real Clusters')
#plt.xlabel('Vmag')
#plt.ylabel('Plx')
#plt.ylim([-10,100])

# Plot KMeans Model Classifications
plt.subplot(1,1,1)
#plt.scatter(X.forehead_w, X.forehead_h,c = colormap[clusters], s = 40)
plt.scatter(X.Vmag, X.Plx, c = colormap[clusters], s=40)
plt.title('KMeans Clusters')
plt.xlabel('Vmag')
plt.ylabel('Plx')
plt.ylim([-10,100])

In [ ]:
scaler = preprocessing.StandardScaler()
scaler.fit(X)
scaled_X = scaler.transform(X)

xs = pd.DataFrame(scaled_X, columns = X.columns)

from sklearn.mixture import GaussianMixture

gmm = GaussianMixture(n_components=2)
gmm_y = gmm.fit_predict(xs)

labels = np.zeros_like(clusters)

for i in range(2):
    cat = (gmm_y == i)
    labels[cat] = mode(y.Targets[cat])[0]
colormap = np.array(['red',  'blue'])  
acc = accuracy_score(y.Targets, labels)
print("Accuracy using GMM = ", acc)
plt.figure(figsize=(20,10))
plt.subplot(2,2,1)
#plt.scatter(X.forehead_w, X.forehead_h, c = colormap[gmm_y], s = 40)
plt.scatter(X.Vmag, X.Plx, c = colormap[gmm_y], s = 40)
plt.subplots_adjust(hspace=0.4, wspace=0.4)
plt.title('GMM Clusters')
plt.xlabel('Vmag')
plt.ylabel('Plx')
plt.ylim([-10,100])

In [ ]:
from scipy.spatial.distance import cdist
distortions = []
inertias = []
mapping1 = {}
mapping2 = {}
K = range(1, 10)
 
for k in K:
    # Building and fitting the model
    kmeanModel = KMeans(n_clusters=k).fit(X)
    kmeanModel.fit(X)
    gmm = GaussianMixture(n_components=2)
    gmm_y = gmm.fit_predict(xs)
 
    distortions.append(sum(np.min(cdist(X, kmeanModel.cluster_centers_,
                                        'euclidean'), axis=1)) / X.shape[0])
    inertias.append(kmeanModel.inertia_)
 
    mapping1[k] = sum(np.min(cdist(X, kmeanModel.cluster_centers_,
                                   'euclidean'), axis=1)) / X.shape[0]
    mapping2[k] = kmeanModel.inertia_

In [ ]:
from sklearn.decomposition import PCA
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt
print(X)
scaled_X = preprocessing.scale(X)
pca = PCA(n_components=2)
px=pca.fit_transform(scaled_X )

colormap = np.array(['red', 'blue'])

# Plot the Original Classifications using Petal features
plt.subplot(1, 1, 1)
plt.scatter(px[:, 0], px[:, 1], c = colormap[y.Targets], s=40)
plt.title('Real Clusters')
plt.xlabel('component 1')
plt.ylabel('component 2')
plt.xlim(-5,10)
plt.ylim(-10,10)

In [ ]:
print(pca.explained_variance_)

In [ ]:
from sklearn.datasets import load_digits
from sklearn.decomposition import FastICA
transformer = FastICA(n_components=2,
         random_state=0)
X_transformed = transformer.fit_transform(X)
X_transformed.shape
plt.subplot(1, 1, 1)
plt.scatter(X_transformed[:, 0], X_transformed[:, 1], c = colormap[y.Targets], s=40)
plt.title('Real Clusters')
plt.xlabel('component 1')
plt.ylabel('component 2')
plt.xlim(-0.2,0.1)


In [ ]:
def kurt(x):
    n = np.shape(x)[0]
    mean = np.sum((x**1)/n) # Calculate the mean
    var = np.sum((x-mean)**2)/n # Calculate the variance
    skew = np.sum((x-mean)**3)/n # Calculate the skewness
    kurt = np.sum((x-mean)**4)/n # Calculate the kurtosis
    kurt = kurt/(var**2)-3

    return kurt
print(kurt(X_transformed))

In [ ]:

from sklearn.random_projection import GaussianRandomProjection

transformer = GaussianRandomProjection(n_components=2)
X_new = transformer.fit_transform(X)
X_new.shape
plt.subplot(1, 1, 1)
plt.scatter(X_new[:, 0], X_new[:, 1], c = colormap[y.Targets], s=40)
plt.title('Real Clusters')
plt.xlabel('component 1')
plt.ylabel('component 2')


In [ ]:
from sklearn import datasets, cluster

agglo = cluster.FeatureAgglomeration(n_clusters=2)
agglo.fit(X)
X_reduced = agglo.transform(X)
X_reduced.shape

In [ ]:
plt.subplot(1, 1, 1)
plt.scatter(X_reduced[:, 0], X_reduced[:, 1], c = colormap[y.Targets], s=40)
plt.title('Real Clusters')
plt.xlabel('component 1')
plt.ylabel('component 2')

In [ ]:
scaled_X = preprocessing.scale(X)
pca = PCA(n_components=2)
pca_x=pca.fit_transform(scaled_X)
transformer = FastICA(n_components=2,
         random_state=0)
ica_x= transformer.fit_transform(X)
transformer = GaussianRandomProjection(n_components=2)
rp_x = transformer.fit_transform(X)
agglo = cluster.FeatureAgglomeration(n_clusters=2)
agglo.fit(X)
fa_x = agglo.transform(X)
ts=['PCA','Fast ICA','Random Projection','Feature Agglomeration']
all_x=[pca_x,ica_x,rp_x,fa_x]
j=0
plt.figure(figsize=(30,25))
for x_i in all_x:
    j=j+1;
    kmeans = KMeans(n_clusters=2)
    clusters1 = kmeans.fit_predict(x_i)
    labels1 = np.zeros_like(clusters1)
    labels2 = np.zeros_like(clusters1)
    
    gmm = GaussianMixture(n_components=2)
    
    
    scaler = preprocessing.StandardScaler()
    scaler.fit(x_i)
    scaled_x_i = scaler.transform(x_i)

    xs = pd.DataFrame(scaled_x_i)    
    clusters2 = gmm.fit_predict(xs)
    for i in range(2):
        cat1 = (clusters1 == i)
        cat2= (clusters2 == i)
        labels1[cat1] = mode(y.Targets[cat1])[0]
        labels2[cat2] = mode(y.Targets[cat2])[0]
    acc1 = accuracy_score(y, labels1)
    acc2 = accuracy_score(y, labels2)
    print("kmeans accuracy:",acc1)
    print("EM accuracy:",acc2)
    
    #draw clustering with comparison to target
    plt.subplot(3,4,j)
    plt.scatter(x_i[:, 0], x_i[:,1],c = colormap[y.Targets], s = 40)
    #plt.scatter(X.Vmag, X.Plx, c = colormap[clusters], s=40)
    plt.title(ts[j-1])
    plt.xlabel('component1')
    plt.ylabel('component2')
    plt.subplot(3,4,j+4)
    plt.scatter(x_i[:, 0], x_i[:,1],c = colormap[labels1], s = 40)
    #plt.scatter(X.Vmag, X.Plx, c = colormap[clusters], s=40)
    t1='K means Clusters, accuracy='+"{:.3f}".format(acc1)
    plt.title(t1)
    plt.xlabel('component1')
    plt.ylabel('component2')
    plt.subplot(3,4,j+8)
    plt.scatter(x_i[:, 0], x_i[:,1],c = colormap[labels2], s = 40)
    #plt.scatter(X.Vmag, X.Plx, c = colormap[clusters], s=40)
    t2='EM Clusters, accuracy='+"{:.3f}".format(acc2)
    plt.title(t2)
    plt.xlabel('component1')
    plt.ylabel('component2')
    

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import f1_score

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

scaled_X = preprocessing.scale(X)
pca = PCA(n_components=2)
pca_x=pca.fit_transform(scaled_X)
transformer1 = FastICA(n_components=2,)
ica_x= transformer1.fit_transform(X)
transformer2 = GaussianRandomProjection(n_components=2)
rp_x = transformer2.fit_transform(X)
agglo = cluster.FeatureAgglomeration(n_clusters=2)
agglo.fit(X)
fa_x = agglo.transform(X)
ts=['PCA','Fast ICA','Random Projection','Feature Agglomeration']
all_x=[pca_x,ica_x,rp_x,fa_x]

h = 0.02  # step size in the mesh

names = [

    "Neural Net",
]

clf=MLPClassifier(alpha=1, max_iter=1000)



datasets = all_x

Fig = plt.figure(figsize=(15,5))
i = 0
# iterate over datasets
for ds in datasets:
    
    i=i+1
    print("i is:",i)
    ax=plt.subplot(1,4,i)
    # preprocess dataset, split into training and test part
    X=ds
    y1=y.Targets.ravel()

    X_train, X_test, y_train, y_test = train_test_split(
        X, y1, test_size=0.4, random_state=42
    )

    x_min, x_max = X[:, 0].min() - 0.5, X[:, 0].max() + 0.5
    y_min, y_max = X[:, 1].min() - 0.5, X[:, 1].max() + 0.5
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))

    # just plot the dataset first
    cm = plt.cm.RdBu
    cm_bright = ListedColormap(["#FF0000", "#0000FF"])


    clf.fit(X_train, y_train)
 
    pred = clf.predict(X_test)
    
    score = f1_score(y_test, pred)
    print(score)

        # Plot the decision boundary. For that, we will assign a color to each
        # point in the mesh [x_min, x_max]x[y_min, y_max].
    if hasattr(clf, "decision_function"):
        Z = clf.decision_function(np.c_[xx.ravel(), yy.ravel()])
    else:
        Z = clf.predict_proba(np.c_[xx.ravel(), yy.ravel()])[:,1]

        # Put the result into a color plot
    Z = Z.reshape(xx.shape)
    ax.contourf(xx, yy, Z, cmap=cm, alpha=0.8)

    # Plot the training points

    ax.scatter(X_train[:, 0], X_train[:, 1], c=y_train, cmap=cm_bright, edgecolors="k")
    # Plot the testing points
    ax.scatter(
        X_test[:, 0],
        X_test[:, 1],
        c=y_test,
        cmap=cm_bright,
        edgecolors="k",
        alpha=0.6,
    )


    ax.set_xticks(())
    ax.set_yticks(())
    




plt.tight_layout()
plt.show()

In [ ]:
print(y.Targets.ravel())

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

scaled_X = preprocessing.scale(X)
pca = PCA(n_components=2)
pca_x=pca.fit_transform(scaled_X)
transformer1 = FastICA(n_components=2,)
ica_x= transformer1.fit_transform(X)
transformer2 = GaussianRandomProjection(n_components=2)
rp_x = transformer2.fit_transform(X)
agglo = cluster.FeatureAgglomeration(n_clusters=2)
agglo.fit(X)
fa_x = agglo.transform(X)
ts=['PCA','Fast ICA','Random Projection','Feature Agglomeration']
all_x=[pca_x,ica_x,rp_x,fa_x]









h = 0.02  # step size in the mesh

names = [

    "Neural Net",
]

clf=MLPClassifier(alpha=1, max_iter=1000)



datasets = all_x

Fig = plt.figure(figsize=(15,5))

i=0
# iterate over datasets
for x_i in datasets:
    


    kmeans = KMeans(n_clusters=2)
    clusters1 = kmeans.fit_predict(x_i)
    labels1 = np.zeros_like(clusters1)
    labels2 = np.zeros_like(clusters1)
    
    gmm = GaussianMixture(n_components=2)
    
    
    scaler = preprocessing.StandardScaler()
    scaler.fit(x_i)
    scaled_x_i = scaler.transform(x_i)

    xs = pd.DataFrame(scaled_x_i)    
    clusters2 = gmm.fit_predict(xs)
    for k in range(2):
        cat1 = (clusters1 == k)
        cat2= (clusters2 == k)
        labels1[cat1] = mode(y.Targets[cat1])[0]
        labels2[cat2] = mode(y.Targets[cat2])[0]
    acc1 = accuracy_score(y, labels1)
    acc2 = accuracy_score(y, labels2)

    
    #add labels1 to x_i

    labels1=np.reshape(labels1,(3642,1))

    x_i_1=np.append(x_i, labels1, axis = 1)
    print(x_i_add.shape)
    labels2=np.reshape(labels2,(3642,1))
    x_i_2=np.append(x_i, labels2, axis = 1)
    
    #rest process is the same as above
    
    
    i=i+1
    print("i is:",i)
    ax=plt.subplot(2,4,i)
    # preprocess dataset, split into training and test part
    X=x_i_1
    y1=y.Targets.ravel()

    X_train, X_test, y_train, y_test = train_test_split(
        X, y1, test_size=0.4, random_state=42
    )

    x_min, x_max = X[:, 0].min() - 0.5, X[:, 0].max() + 0.5
    y_min, y_max = X[:, 1].min() - 0.5, X[:, 1].max() + 0.5
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))

    # just plot the dataset first
    cm = plt.cm.RdBu
    cm_bright = ListedColormap(["#FF0000", "#0000FF"])


    clf.fit(X_train, y_train)
 
    pred = clf.predict(X_test)
    
    score = f1_score(y_test, pred)
    print(score)

        # Plot the decision boundary. For that, we will assign a color to each
        # point in the mesh [x_min, x_max]x[y_min, y_max].


    # Plot the training points

    ax.scatter(X_train[:, 0], X_train[:, 1], c=y_train, cmap=cm_bright, edgecolors="k")
    # Plot the testing points
    ax.scatter(
        X_test[:, 0],
        X_test[:, 1],
        c=y_test,
        cmap=cm_bright,
        edgecolors="k",
        alpha=0.6,
    )


    ax.set_xticks(())
    ax.set_yticks(())


    
    
    ax=plt.subplot(2,4,i+4)
    # preprocess dataset, split into training and test part
    X=x_i_2
    y1=y.Targets.ravel()

    X_train, X_test, y_train, y_test = train_test_split(
        X, y1, test_size=0.4, random_state=42
    )

    x_min, x_max = X[:, 0].min() - 0.5, X[:, 0].max() + 0.5
    y_min, y_max = X[:, 1].min() - 0.5, X[:, 1].max() + 0.5
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))

    # just plot the dataset first
    cm = plt.cm.RdBu
    cm_bright = ListedColormap(["#FF0000", "#0000FF"])


    clf.fit(X_train, y_train)
 
    pred = clf.predict(X_test)
    
    score = f1_score(y_test, pred)
    print(score)

        # Plot the decision boundary. For that, we will assign a color to each
        # point in the mesh [x_min, x_max]x[y_min, y_max].


    # Plot the training points

    ax.scatter(X_train[:, 0], X_train[:, 1], c=y_train, cmap=cm_bright, edgecolors="k")
    # Plot the testing points
    ax.scatter(
        X_test[:, 0],
        X_test[:, 1],
        c=y_test,
        cmap=cm_bright,
        edgecolors="k",
        alpha=0.6,
    )


    ax.set_xticks(())
    ax.set_yticks(())    





plt.tight_layout()
plt.show()
    
    
    